In [1]:
import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    if "google.colab" in sys.modules:
        !pip install -q pyomo
    else:
        !conda install -c conda-forge pyomo
    assert(shutil.which("pyomo"))
    
if not (shutil.which("glpsol") or os.path.isfile("glpsol")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq glpk-utils
    else:
        try:
            !conda install -c conda-forge glpk 
        except:
            pass
    assert(shutil.which("glpsol") or os.path.isfile("glpsol"))

import pyomo.environ as pe

# Blocs 1 & 2:  données et paramètres d'entrée
Nous avons deux ensembles de données différents, mais leur structure est la même. L'ensemble de données #1 est le même que dans l'article, tandis que l'ensemble de données #2 est un ensemble de données plus grand.

Ces jeux de données contiennent des données sur les aspects suivants:
1.   **Centre de distribution**: chaque ensemble de données contient un ensemble de centres de distribution $I \ni i$ (`listFC`), dont chacun a un niveau de stock donné $X_i$ (`inventoryFC`). Les clients peuvent passer une commande pour un seul article ou pour plusieurs articles, mais si aucun centre de distribution ne dispose de tous les articles disponibles, le détaillant doit recourir à des livraisons fractionnées, ce qui affecte évidemment les frais d'expédition. Par conséquent, notre modèle d'optimisation doit tenir compte de la probabilité que le centre de distribution ait les autres articles en commande $\rho_i$ (`probMultiAvailability`).
2.   **Région client**: chaque zone client $J \ni j$ (`listRegion`) doit être associée à une certaine demande au cours des prochains $\tau$ jours $D_j$ (`demandValue`) pour le client $j$. Chaque zone est aussi associée à une proportion de commandes contenant plusieurs articles $\lambda_j$ (`probMultiItem`). 
2.   **Coût d'expédition**: `costSingle[i,j]` désigne le coût d'une seule livraison depuis le centre de distribution $i$ à la zone client $j$ ou $c_{ij}$. Veuillez noter que pour les données #1, puisqu'il n'y a qu'une seule zone client (Kansas), il n'y a qu'un seul coût à partir de chaque centre de distribution $i$. `avgNoMultiItem` représente le *nombre moyen d'articles dans une commande multi-articles*. Cette information est nécessaire pour calculer le coût d'expédition moyen par article pour les commandes multi-articles. Plus précisement, $\omega = 1/\mbox{avgNoMultiItem}$.

Les variables précédentes correspondent à celles présenter dans les diapositives et diffèrent quelque peu des variables de l'article.

In [2]:
DATA = 1  # choisir 1 ou 2


if DATA == 1:  # Données #1 (comme l'article)
    # Données du centre de distribution (FC)
    listFC = ['Utah', 'Nevada']
    inventoryFC = dict(zip(listFC, [5, 20]))
    probMultiAvailability = dict(zip(listFC, [0.5, 0.2]))
    # Données de région client
    listRegion = ['Kansas']
    demandValue = dict(zip(listRegion, [2 * 10]))  # demand journalière=2, tau=10 
    probMultiItem = dict(zip(listRegion, [0.75]))
    # Données de coût d'expédition
    costSingle = [[9],
                  [12]]
    # Nombre moyen d'articles dans une commande multi-articles
    avgNoMultiItem = 3.0
elif DATA == 2:  # Données #2
    # Données du centre de distribution (FC)
    listFC = ['Delta-BC', 'Brampton-ON', 'Ottawa-ON']
    inventoryFC = dict(zip(listFC, [37, 85, 25]))
    probMultiAvailability = dict(zip(listFC, [0.32, 0.45, 0.17]))
    # Données de région client
    listRegion = ['Toronto-ON', 'Montreal-QC', 'Calgary-AB', 'Vancouver-BC']
    demandValue = dict(zip(listRegion, [45, 27, 15, 33]))
    probMultiItem = dict(zip(listRegion, [0.73, 0.68, 0.54, 0.64]))
    # Données de coût d'expédition
    costSingle = [[24.5, 25.5, 18.1, 12.3],
                  [13.6, 17.5, 22.8, 23.6],
                  [18.1, 14.1, 21.1, 22.8]]
    # Nombre moyen d'articles dans une commande multi-articles
    avgNoMultiItem = 2.5
else:
    raise ValueError("DATA doit être défini à 1 ou 2")

    
# Vérification des données (clés et tailles)
assert listFC == list(inventoryFC.keys())
assert listFC == list(probMultiAvailability.keys())
assert listRegion == list(demandValue.keys())
assert listRegion == list(probMultiItem.keys())
assert len(listFC) == len(costSingle)
for i in costSingle:
    assert len(listRegion) == len(i)

    
# Préparez les frais d'expédition ci-dessus en format dictionnaire
costSingleDict = {}
for i_, i in enumerate(listFC):  # i_ contient l'index et i contient le FC
    for j_, j in enumerate(listRegion):  # j_ contient l'index et j contient la région
        costSingleDict[i, j] = costSingle[i_][j_]
# Nous pouvons calculer la remise d'expédition multi-articles $\omega$
shippingDiscount = 1 / avgNoMultiItem


# Affichage des données
print('**Données**\n')

print(f'listFC (I): {listFC}')
print(f'inventoryFC (X_i): {inventoryFC}')
print(f'probMultiAvailability (rho_i): {probMultiAvailability}\n')

print(f'listRegion (J): {listRegion}')
print(f'demandValue (D_j): {demandValue}')
print(f'probMultiItem (lambda_j): {probMultiItem}\n')

print(f'costSingle (c_ij):\n {costSingle}')
print(f'costSingleDict: {costSingleDict}\n')

print(f'avgNoMultiItem: {avgNoMultiItem}')
print(f'shippingDiscount (omega): {shippingDiscount}')

**Données**

listFC (I): ['Utah', 'Nevada']
inventoryFC (X_i): {'Utah': 5, 'Nevada': 20}
probMultiAvailability (rho_i): {'Utah': 0.5, 'Nevada': 0.2}

listRegion (J): ['Kansas']
demandValue (D_j): {'Kansas': 20}
probMultiItem (lambda_j): {'Kansas': 0.75}

costSingle (c_ij):
 [[9], [12]]
costSingleDict: {('Utah', 'Kansas'): 9, ('Nevada', 'Kansas'): 12}

avgNoMultiItem: 3.0
shippingDiscount (omega): 0.3333333333333333


# Bloc 3: création du modèle d'optimisation

## Bloc 3.1: déclarations de variables

Comme nous l'avons appris lors de la séance 9, un modèle d'optimisation comprend (1) des variables de décision, (2) une fonction objectif et (3) des contraintes.

Nos variables de décision incluent `model.x`, `model.y` et `model.w`. Ces variables sont toutes non-négatives et indiquent le flux du centre de distribution $i$ vers la région client $j$. En particulier,

- `model.x[i, j]` (ou $x_{ij}$) désigne la variable de décision pour **l'expédition de plusieurs articles en un seul envoi** du centre de distribution `i` vers une région client `j`. Le coût unitaire de ce flux est $\omega c_{ij}$;

- `model.y[i, j]` (ou $y_{ij}$) désigne la variable de décision pour **l'expédition de plusieurs articles en plusieurs envois** du centre de distribution `i` vers une région client `j`. Le coût unitaire de ce flux est $2\omega c_{ij}$ (deux envois sont utilisés si la commande ne peut pas être expédiée en un seul envoi);

- `model.w[i, j]` (ou $w_{ij}$) désigne la variable de décision pour **l'expédition d'un seul article** du centre de distribution `i` vers une région client `j`. Le coût unitaire de ce flux est $c_{ij}$.

Sur la dernière ligne de code de la prochaine cellule, nous créons également un objet pour stocker les coûts ombres (variables duales). Ceci est nécessaire pour éviter de résoudre le programme linéaire (PL) plusieurs fois.

In [3]:
model = pe.ConcreteModel()

# Variables
model.x = pe.Var(listFC, listRegion, within=pe.NonNegativeReals)
model.y = pe.Var(listFC, listRegion, within=pe.NonNegativeReals)
model.w = pe.Var(listFC, listRegion, within=pe.NonNegativeReals)

# Création d'un objet pour accéder aux coûts marginaux
model.dual = pe.Suffix(direction=pe.Suffix.IMPORT)

## Bloc 3.2: ajout d'une fonction objectif
Comme dans les diapositives, nous voulons minimiser la fonction objectif suivante

$$\text{minimiser}_{x,y,w} \sum_{i\in I} \sum_{j\in J}\left(  c_{ij} w_{ij} + \omega  c_{ij} x_{ij} + 2\omega  c_{ij} y_{ij} \right)$$

où $c_{ij}=$ `costSingleDict[i, j]` et $\omega=$ `shippingDiscount`.

In [4]:
obj_expr = sum(
    costSingleDict[i, j] * model.w[i, j] + 
    shippingDiscount * costSingleDict[i, j] * model.x[i, j] +
    2 * shippingDiscount * costSingleDict[i, j] * model.y[i, j]
    for i in listFC for j in listRegion)
model.obj = pe.Objective(expr=obj_expr, sense=pe.minimize)
model.obj.pprint()

obj : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : minimize : 9*w[Utah,Kansas] + 3.0*x[Utah,Kansas] + 6.0*y[Utah,Kansas] + 12*w[Nevada,Kansas] + 4.0*x[Nevada,Kansas] + 8.0*y[Nevada,Kansas]


## Bloc 3.3: ajout des contraintes

### Contraintes 1: disponibilité de stock au centre de distribution $i$

Ces contraintes garantissent que la demande totale (des commandes) affectée au centre de distribution `i` est inférieure ou égale à ses stocks.

Ces contraintes correspondent à

$$\sum_{j\in J} \left( w_{ij} + x_{ij} + y_{ij} \right) \leq X_i,\quad \forall i\in I$$

où $X_i=$ `inventoryFC[i]`.

In [5]:
model.inventoryOnHand = pe.ConstraintList()
for i in listFC:
    const_expr = sum(model.w[i, j] + model.x[i, j] + model.y[i, j] 
                     for j in listRegion) <= inventoryFC[i] 
    model.inventoryOnHand.add(expr=const_expr)
model.inventoryOnHand.pprint()

# Code équivalent:
# def inventoryOnHand_rule(m, i):
#     return sum(m.w[i, j] + m.x[i, j] + m.y[i, j] 
#                for j in listRegion) <= inventoryFC[i] 
# model.inventoryOnHand = pe.Constraint(listFC, rule=inventoryOnHand_rule)
# model.inventoryOnHand.pprint()

inventoryOnHand : Size=2, Index={1, 2}, Active=True
    Key : Lower : Body                                                   : Upper : Active
      1 :  -Inf :       w[Utah,Kansas] + x[Utah,Kansas] + y[Utah,Kansas] :   5.0 :   True
      2 :  -Inf : w[Nevada,Kansas] + x[Nevada,Kansas] + y[Nevada,Kansas] :  20.0 :   True


### Contraintes 2: demande future de commandes avec un seul article dans la région $j$

Ces contraintes garantissent que la demande d'articles uniques dans la région client $j$ est satisfaite.

Ces contraintes correspondent à

$$\sum_{i \in I} w_{ij} = D_j \left( 1-\lambda_j \right),\quad \forall j\in J$$

où $D_j=$ `demandValue[j]` et $\lambda_j=$ `probMultiItem[j]`.

In [6]:
model.demandSingle = pe.ConstraintList()
for j in listRegion:
    const_expr = sum(model.w[i, j] for i in listFC) == demandValue[j] * (1 - probMultiItem[j])
    model.demandSingle.add(expr=const_expr)
model.demandSingle.pprint()

# Code équivalent:
# def demandSingle_rule(m, j):
#     return sum(m.w[i, j] for i in listFC) == demandValue[j] * (1 - probMultiItem[j])
# model.demandSingle = pe.Constraint(listRegion, rule=demandSingle_rule)
# model.demandSingle.pprint()

demandSingle : Size=1, Index={1}, Active=True
    Key : Lower : Body                              : Upper : Active
      1 :   5.0 : w[Utah,Kansas] + w[Nevada,Kansas] :   5.0 :   True


### Contraintes 3: demande future de commandes multi-articles dans la région $j$

Ces contraintes garantissent que la demande multi-articles dans la région client $j$ est satisfaite.

Ces contraintes correspondent à
$$\sum_{i \in I}  \left( x_{ij} + y_{ij} \right) = D_j \lambda_j,\quad \forall j\in J $$

où $D_j=$ `demandValue[j]` et $\lambda_j=$ `probMultiItem[j]`.

In [7]:
model.demandMulitiple = pe.ConstraintList()
for j in listRegion:
    const_expr = sum(model.x[i, j] + model.y[i, j]
                     for i in listFC) == demandValue[j] * probMultiItem[j]
    model.demandMulitiple.add(expr=const_expr)
model.demandMulitiple.pprint()

# Code équivalent:
# def demandMulitiple_rule(m, j):
#     return sum(m.x[i, j] + m.y[i, j]
#                for i in listFC) == demandValue[j] * probMultiItem[j]
# model.demandMulitiple = pe.Constraint(listRegion, rule=demandMulitiple_rule)
# model.demandMulitiple.pprint()

demandMulitiple : Size=1, Index={1}, Active=True
    Key : Lower : Body                                                                  : Upper : Active
      1 :  15.0 : x[Utah,Kansas] + y[Utah,Kansas] + x[Nevada,Kansas] + y[Nevada,Kansas] :  15.0 :   True


### Contraintes 4: nombre maximum de commandes multi-articles expédiées en un seul envoi de $i$ vers $j$

Il convient de noter que la livraison non-fractionnée pour une commande multi-articles du centre de distribution $i$ vers la région client $j$ est effectuée uniquement lorsque $i$ a les autres articles en commande. Ces contraintes garantissent qu'on ne dépasse pas le nombre **d'expédition de plusieurs articles en un seul envoi** possible.

Ces contraintes correspondent à

$$ x_{ij} \leq D_j \lambda_j \rho_i,\quad \forall i\in I, j\in J $$

où $D_j=$ `demandValue[j]`, $\lambda_j=$ `probMultiItem[j]` et $\rho_i=$ `probMultiAvailability[i]`.

In [8]:
model.maxMultiShipment = pe.ConstraintList()
for i in listFC:
    for j in listRegion:
        const_expr = model.x[i, j] <= demandValue[j] * probMultiItem[j] * probMultiAvailability[i]
        model.maxMultiShipment.add(expr=const_expr)
model.maxMultiShipment.pprint()

# Code équivalent:
# def maxMultiShipment_rule(m, i, j):
#     return m.x[i, j] <= demandValue[j] * probMultiItem[j] * probMultiAvailability[i]
# model.maxMultiShipment = pe.Constraint(listFC, listRegion, rule=maxMultiShipment_rule)
# model.maxMultiShipment.pprint()

maxMultiShipment : Size=2, Index={1, 2}, Active=True
    Key : Lower : Body             : Upper : Active
      1 :  -Inf :   x[Utah,Kansas] :   7.5 :   True
      2 :  -Inf : x[Nevada,Kansas] :   3.0 :   True


Finalement, vous pouvez afficher le modèle entier, si vous le désirez, en enlevant le `#` suivant.

In [9]:
#model.pprint()

# Bloc 4: solution et interprétation
Enfin, nous appelons le solveur et obtenons la solution optimale. On remarque pour les données #1 que la solution optimale est la même que dans l'article.

In [10]:
# Résolution du modèle
pe.SolverFactory('glpk').solve(model)
model.display()

Model unknown

  Variables:
    x : Size=2, Index={Utah, Nevada}*{Kansas}
        Key                  : Lower : Value : Upper : Fixed : Stale : Domain
        ('Nevada', 'Kansas') :     0 :   3.0 :  None : False : False : NonNegativeReals
          ('Utah', 'Kansas') :     0 :   5.0 :  None : False : False : NonNegativeReals
    y : Size=2, Index={Utah, Nevada}*{Kansas}
        Key                  : Lower : Value : Upper : Fixed : Stale : Domain
        ('Nevada', 'Kansas') :     0 :   7.0 :  None : False : False : NonNegativeReals
          ('Utah', 'Kansas') :     0 :   0.0 :  None : False : False : NonNegativeReals
    w : Size=2, Index={Utah, Nevada}*{Kansas}
        Key                  : Lower : Value : Upper : Fixed : Stale : Domain
        ('Nevada', 'Kansas') :     0 :   5.0 :  None : False : False : NonNegativeReals
          ('Utah', 'Kansas') :     0 :   0.0 :  None : False : False : NonNegativeReals

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : 

Il est possible d'accéder les valeurs des variables en utilisant le paramètre `value`.

In [11]:
print('**x**')
for index in model.x:
    print(f"x[{index}]: {model.x[index].value}")

print('\n**y**')
for index in model.y:
    print(f"y[{index}]: {model.y[index].value}")

print('\n**w**')
for index in model.w:
    print(f"w[{index}]: {model.w[index].value}")

**x**
x[('Utah', 'Kansas')]: 5.0
x[('Nevada', 'Kansas')]: 3.0

**y**
y[('Utah', 'Kansas')]: 0.0
y[('Nevada', 'Kansas')]: 7.0

**w**
w[('Utah', 'Kansas')]: 0.0
w[('Nevada', 'Kansas')]: 5.0


Afin de déterminer rapidement le coût à venir approximatif sans résoudre plusieurs fois le modèle PL ci-dessus, nous pouvons utiliser les coûts ombres (c.-à-d., les variables duales). Le code ci-dessous affiche les coûts ombres de toutes les contraintes. Seuls les coûts ombres de la contrainte `inventoryOnHand` sont utilisés pour calculer le coût à venir approximatif, c.-à-d., 

$$ C_{k}({\bf X_k})=\min_{i\in I}\left({c_{ik}+C_{k+1}({\bf X_{k+1}})}\right) \approx \min_{i\in I}\left({c_{ik}+C_{k+1}({\bf X_{k}})-\pi_{i}}\right)$$

où ${\bf X_k}$ (${\bf X_{k+1}}$) correspond aux stocks à la période $k$ ($k+1$) pour tous les centres de distribution dans $I$, $c_{ik}$ correspond au coût d'envoyer la commande de $i$ à la période $k$, et $\pi_i$ correspond à la variation du coût total si le stock de $i$ augmente d'une unité. Donc, $\pi_i$ correspond à la variation du coût à venir si `inventoryFC[i]` augmente d'une unité; on fait $-\pi_i$ car on veut obtenir l'effet de diminuer les stocks d'une unité si on choisit $i$.

In [12]:
# Obtention un coût réduit pour chaque contrainte
model.dual.display()

dual : Direction=IMPORT, Datatype=FLOAT
    Key                 : Value
     demandMulitiple[1] :   8.0
        demandSingle[1] :  12.0
     inventoryOnHand[1] :  -5.0
     inventoryOnHand[2] :   0.0
    maxMultiShipment[1] :   0.0
    maxMultiShipment[2] :  -4.0


In [13]:
duals_dict = {str(key):model.dual[key] for key in model.dual.keys()}
duals_dict

{'inventoryOnHand[1]': -5.0,
 'inventoryOnHand[2]': 0.0,
 'demandSingle[1]': 12.0,
 'demandMulitiple[1]': 8.0,
 'maxMultiShipment[1]': 0.0,
 'maxMultiShipment[2]': -4.0}